In [20]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_10.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    return max_corr if max_corr >= threshold else 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename):
    submatrix = pd.DataFrame(index=columns, columns=columns)
    for col1, col2 in product(columns, repeat=2):
        submatrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    submatrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix{filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


#filename_sub = "10_sub_mat_3.csv"
#compute_submatrix(data, group_3, filename_sub)

filename_cross = "10_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross)


submatrix10_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,0.566541,0.609671,0.344673,0.442447,0.544804,0.698712,0.721465,0.31061,0.618777,0.675103,...,0.351929,0.603587,0.585372,0.599055,0.435001,0.522527,0.622141,0.606714,0.537073,0.594399
21085.0,0.512049,0.463394,0.316404,0.464087,0.534701,0.603433,0.526931,0.416565,0.579438,0.563965,...,0.208434,0.542829,0.421424,0.494936,0.463728,0.366048,0.43423,0.474645,0.399472,0.348861
21087.0,0.390623,0.421366,0.223721,0.445119,0.573472,0.521666,0.440009,0.396278,0.58321,0.49852,...,0.235827,0.409844,0.369216,0.451481,0.303163,0.404915,0.368234,0.49226,0.414664,0.292868
21089.0,0.490795,0.439518,0.487087,0.354327,0.465842,0.580267,0.620113,0.336369,0.536144,0.569587,...,0.438658,0.551398,0.433725,0.468501,0.462667,0.49423,0.512882,0.514182,0.617836,0.637782
21091.0,0.622349,0.53081,0.355193,0.440604,0.568043,0.636604,0.676287,0.276923,0.625703,0.711844,...,0.278438,0.553173,0.506636,0.591549,0.569034,0.412421,0.667417,0.594925,0.646682,0.452299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.530016,0.347619,0.348019,0.389109,0.293822,0.408671,0.369824,0.244822,0.311429,0.407147,...,0.34639,0.335153,0.381343,0.317332,0.342926,0.363419,0.235238,0.436403,0.326623,0.293834
38091.0,0.368473,0.389234,0.290999,0.218164,0.347757,0.288353,0.350456,0.176007,0.222735,0.315966,...,0.280018,0.351199,0.26185,0.273493,0.258466,0.273587,0.224303,0.234831,0.343176,0.239268
38093.0,0.583048,0.550364,0.350399,0.429659,0.471684,0.622485,0.554357,0.197404,0.485992,0.613033,...,0.294906,0.67839,0.569903,0.548801,0.468389,0.571244,0.385585,0.579166,0.487886,0.514549
38095.0,0.452374,0.448429,0.22802,0.285699,0.443028,0.485314,0.486114,0.177579,0.407623,0.476485,...,0.296122,0.507743,0.352246,0.467427,0.387249,0.423446,0.36722,0.470314,0.447014,0.389133
